## News :D



In [28]:
%load_ext autoreload
%autoreload 2

import requests
import re 
import itertools
from bs4 import BeautifulSoup # Imports bs4
import wget # Imports wget
import articles
import sitemaps

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Data

### Data formats

CNN has links by month

NYT has links by day

BI has links by month

TH has links by day

NYP has links by day

DM has links by day

Fox has links by page num

### Thoughts on how to organize the data 

we use a pandas dataframe organized like this

| Dates | CNN | NYT | BI | TH | NYP | DM | FOX |
| ----------- | ------------------------------------------ | ------- | ------- | ------- | ------- | ------- | ------- |
| 2014-01-01  | list of tuples with article link & title   | ... | ... | ... | ... | ... | ... |
| 2014-01-02  | ... | ... | ... | ... | ... | ... | ... |
| 2014-01-03  | ... | ... | ... | ... | ... | ... | ... |
| 2014-01-04  | ... | ... | ... | ... | ... | ... | ... |
| ...         | ... | ... | ... | ... | ... | ... | ... |
| 2024-12-31  | ... | ... | ... | ... | ... | ... | ... |

### Code to get data
CNN:
```html
<li>
    <span class="date">2024-01-31</span>
    <span class="sitemap-link">
        <a href="https://www.cnn.com/tech/live-news/meta-x-discord-tiktok-snap-chiefs-testimony-senate/index.html">
            Mark Zuckerberg apologizes to families over social media harms in contentious Senate hearing
        </a>
    </span>
</li>
```

NYT:
```html
<li>
    <a href="https://www.nytimes.com/2023/12/31/business/dealbook/pga-tour-saudi-deal-deadline.html">
        PGA Tour and Saudi-Backed LIV Extend Deadline to Finalize Deal
    </a>
</li>
```

BI:
```html
<p>
    <a href="https://www.businessinsider.com/feud-capote-vs-swans-ann-woodward-suicide-death-real-story-2024-1">
        The real story behind the suicide of American socialite Ann Woodward in &#39;Feud: Capote vs. The Swans&#39;
    </a>
    <br>2024-02-01T03:31:01.68Z
</p>
```

NYP:
```html
<h3 class="story__headline headline headline--archive">
    <a href="https://nypost.com/2024/01/01/sports/og-anunoby-trending-toward-the-perfect-piece-for-knicks/" rel="" target="_self">
		Knicks&#039; big trade acquisition already looks like the &#039;perfect fit&#039; 												
    </a>
</h3>
```

DM:
```html
<a href="/wires/ap/article-12917573/Christian-McCaffrey-miss-49ers-regular-season-finale-return-playoffs.html" title="">
    Christian McCaffrey will miss the 49ers regular-season finale but should return for playoffs
</a>
```

FOX:
```html
<loc>
    https://www.foxnews.com/opinion/why-the-department-of-education-is-going-to-rip
</loc>
<lastmod>
    2025-03-12T05:00:54-04:00
</lastmod>
```

In [29]:
[cnn_dict, nyt_dict, bi_dict, nyp_dict, dm_dict, fox_links] = sitemaps.sitemaps()

In [83]:
start_year = 2020
end_year = 2020
start_month = 1
end_month = 1
news = [1,1,1,0,1,0] # determines which news sources we scrape (for testing)       0 = don't   1 = do       CNN, NYT, BI, NYP, DM, FOX

[cnn_articles, nyt_articles, bi_articles, nyp_articles, dm_articles, fox_articles] = articles.articles(news,start_year,end_year,start_month,end_month)

In [84]:
# Pandas Testing area
import pandas as pd
import numpy as np

mday = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

news_names = ['CNN', 'NYT', 'BI', 'NYP', 'DM', 'FOX']
temp_news_names = ['CNN', 'NYT', 'BI', 'NYP', 'DM', 'FOX']
news_data = {}
example_data = {}

for year in range(start_year,end_year+1):
    for month in range(start_month,end_month+1):
        for day in range(mday[month-1]):
            news_data[f"{year}/{str(month).zfill(2)}/{str(day+1).zfill(2)}"] = [
                cnn_articles[f"{year}-{str(month).zfill(2)}-{str(day+1).zfill(2)}"],
                nyt_articles[f"{year}-{str(month).zfill(2)}-{str(day+1).zfill(2)}"],
                bi_articles[f"{year}-{str(month).zfill(2)}-{str(day+1).zfill(2)}"],
                'N/A',
                dm_articles[f"{year}-{str(month).zfill(2)}-{str(day+1).zfill(2)}"],
                'N/A'
            ]
            example_data[f"{year}/{str(month).zfill(2)}/{str(day+1).zfill(2)}"] = [
                'list',
                'list',
                'list',
                'list',
                'list',
                'list'
            ]

test_frame = pd.DataFrame(news_data, index=temp_news_names).T
example_test_frame = pd.DataFrame(example_data, index=news_names).T

print(test_frame)
print(example_test_frame)

                                                          CNN  \
2020/01/01  [(Princeton University is hanging a series of ...   
2020/01/02  [(15 top-rated 2020 planners on Amazon to keep...   
2020/01/03  [(Here's how to help Australia bushfire victim...   
2020/01/04  [(Trump threatens the stability of the world, ...   
2020/01/05  [(Awkwafina makes Golden Globes history, https...   
2020/01/06  [(TESS mission spots Earth-size potentially ha...   
2020/01/07  [(Allergic to dogs? It may only be the males, ...   
2020/01/08  [(Estranged husband, his ex-girlfriend and his...   
2020/01/09  [(Estranged husband of Connecticut mom Jennife...   
2020/01/10  [(She went head-to-head with an oil company fi...   
2020/01/11  [(Podium order announced for CNN/Des Moines Re...   
2020/01/12  [(The majority of women feel relief, not regre...   
2020/01/13  [(An Alabama county is preserving a jail cell ...   
2020/01/14  [(Davos VIPs are worried about the climate cri...   
2020/01/15  [(US countert